### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Lupus_(Systemic_Lupus_Erythematosus)/GSE200306'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Check for gene expression data availability
is_gene_available = True  # The dataset contains transcriptomic analysis indicating gene expression data

# Step 2: Data Availability and Type Conversion

# Trait (Lupus)
trait_row = 0  # Assume row 0 contains information related to lupus classes
def convert_trait(value):
    value = value.split(':')[-1].strip().lower()
    if 'healthy control' in value:
        return 0
    elif 'class' in value:
        return 1
    return None

# Age
age_row = 2  # Row 2 contains age information
def convert_age(value):
    try:
        if isinstance(value, str):
            value = float(value.split(':')[-1].strip())
        return value
    except ValueError:
        return None

# Gender
gender_row = 1  # Row 1 contains gender information
def convert_gender(value):
    if value is None:
        return None
    value = value.split(':')[-1].strip().upper()
    if value == 'M':
        return 1
    elif value == 'F':
        return 0
    return None

# Step 3: Save Metadata
save_cohort_info('GSE200306', './preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Lupus_(Systemic_Lupus_Erythematosus)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/trait_data/GSE200306.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# Requires gene mapping
if requires_gene_mapping:
    # STEP6
    # Substep 1
    identifier_key = 'ID'
    gene_symbol_key = 'Official Full Name'

    # Substep 2
    gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

    # Substep 3
    gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


from utils.preprocess import *


### Step 7: Data Normalization and Merging

In [ ]:
# Ensure previous steps define gene_data
# Load gene_data from previously saved CSV if necessary, otherwise proceed from previous steps
try:
    gene_data = pd.read_csv('./preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE200306.csv', index_col=0)
except FileNotFoundError:
    pass  # Handle the condition where the file might not be available

# Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE200306.csv'
normalized_gene_data.to_csv(gene_csv_path)

# Ensure selected_clinical_data is defined. Assume it's obtained from previous steps.
selected_clinical_data = pd.read_csv('./preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/trait_data/GSE200306.csv', index_col=0)

# Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# Determine whether the trait and some demographic attributes in the data is severely biased,
# and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Lupus_(Systemic_Lupus_Erythematosus)')

# Save the cohort information.
save_cohort_info('GSE200306', './preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/GSE200306.csv'
    unbiased_merged_data.to_csv(csv_path)
